In [16]:
import pandas as pd
import numpy as np
import scipy

import os
import datetime

### Read in Data

In [33]:
# Read in the bid-ask data from WRDS
bid_ask_df = pd.read_sas('../Data/bid_ask_data.sas7bdat')
print(bid_ask_df.shape)
bid_ask_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../Data/bid_ask_data.sas7bdat'

In [ ]:
len(bid_ask_df.PERMNO.unique())

In [2]:
# Read in the CCI data
CCI_df = pd.read_csv('../Data/CCI.csv')
print(CCI_df.shape)
CCI_df.head()

(13203, 8)


,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,NLD,CCI,AMPLITUD,LTRENDIDX,M,1991-01,99.75915,NaN
1,NLD,CCI,AMPLITUD,LTRENDIDX,M,1991-02,99.74998,NaN
2,NLD,CCI,AMPLITUD,LTRENDIDX,M,1991-03,99.88835,NaN
3,NLD,CCI,AMPLITUD,LTRENDIDX,M,1991-04,99.95676,NaN
4,NLD,CCI,AMPLITUD,LTRENDIDX,M,1991-05,100.01380,NaN


In [3]:
# Read in the ICS data
ICS_df = pd.read_csv('../Data/ICS.csv')
print(ICS_df.shape)
ICS_df.head()

(607, 3)


,Month,YYYY,ICS_ALL
0,November,1952,86.2
1,February,1953,90.7
2,August,1953,80.8
3,November,1953,80.7
4,February,1954,82.0


### Helper Functions

In [49]:
def split_date_field(df, date_field):
    '''
    Extracts year and month columns from date column
    
    Return: df with month and year columns
    '''
    new_df = df
    
    split_date = new_df[date_field].str.split('-', n=1, expand=True)
    print(split_date[0])
    
    new_df['Year'] = split_date[0].astype(int)
    new_df['Month'] = split_date[1].astype(int)
    
    return new_df
    
def get_integer_month(df, month_field, short_names=False):
    '''
    Get months as integers from string month field
    
    Return: df with updated month field to have integer values
    '''
    
    new_df = df
    
    # Check for integer months
    if pd.api.types.is_integer_dtype(df[month_field]):
        print("Month field already in integer form.")
        return df
    
    # If not integer months, convert months to integers
    if short_names:
        new_df[month_field] = new_df[month_field].apply(lambda x: datetime.datetime.strptime(x, "%b").month)
    
    else:
        new_df[month_field] = new_df[month_field].apply(lambda x: datetime.datetime.strptime(x, "%B").month)

    return new_df
    
    
def interpolate(df, year_field, month_field, interpolate_field):
    '''
    Interpolates the interpolate_field values for each day between month-ends by repeating the previous value
    
    Return: df with interpolated values
    '''
    new_df = df
    
    return new_df

In [ ]:
def bid_ask_estimate():
    # Question for Prof. Kramer: to compute beta (E[(ln(H_t/L_t)^2 + ln(H_t+1/L_t+1)^2)]) do we just take the mean?
    
    # beta = E{sum(j=0,1)[ln(H_(t+j)/L_(t+j))]^2} where H_(t) is the observed high price on day t (similarly for L)
    beta = ...
    
    # gamma = [ln(H_(t, t+1)/L_(t, t+1))]^2 where H_(t, t+1) is the high price over days t and t+1 (similarly for L)
    gamma = (np.log(high/low))**2
    
    # alpha = ((2beta)^0.5-beta^0.5)/(3-2*2^0.5) - (gamma/(3-2*2^0.5))^0.5
    alpha = (np.sqrt(2*beta)-np.sqrt(beta))/(3-2*np.sqrt(2))-np.sqrt(gamma/(3-2*np.sqrt(2)))
    
    # high-low spread estimate S = 2(e^alpha - 1)/(1+e^alpha)
    S = 2*(np.exp(alpha)-1)/(1+np.exp(alpha))
    
    return S

### Pre-process the Data